## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

##### train_err_data , train_quality_data, train_problem_data
###### user_id ,time ,fwver ,model_nm ,errtype ,errcode  
###### user_id ,time ,fwver ,quality_0 ~ quality_12
###### user_id ,time 

train_err
models per user
total user : 15000
1    14297  : 0.95
2      702  : 0.046 
3        1  : ...

errtype :  1부터 42까지 29제외
month range : (10,11,12) #year 모두 2020

train_qual
userid 8281 (of 15000)

train_prob
total  5000
rows   5429


# 1. Library Import

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)

In [20]:
from time import sleep as sl

In [28]:
from tqdm import tnrange

In [37]:
for x in tnrange(3,desc="test"):
    sl(0.5)
    print("ta")
    

test:   0%|          | 0/3 [00:00<?, ?it/s]

ta
ta
ta


In [3]:
PATH = './data/'

# 2. 학습 데이터 생성

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [126]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qual = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
display(train_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


###### int64 ->Object(str) -> timestamp(date64) (->perid or something)

In [181]:
pd.Timestamp(str(20201130172625))

Timestamp('2020-11-30 17:26:25')

In [197]:
## comparing each performance
import timestamp as tt
first = t()
for x in tqdm(range(200),desc="test"):
    train_err[:10].time.astype('str').apply(lambda x : pd.Timestamp(x))
last = t()- first
print(last)



first = t()
for x in tqdm(range(200)):
    train_err[:10].time.apply(lambda x : pd.Timestamp(str(x)) )
last = t()- first
print(last)

In [201]:
train_err.time.astype('str').apply(lambda x : pd.Timestamp(x) )

KeyboardInterrupt: 

In [ ]:
pd.to_datetime(train_err.time.astype('str'))

In [209]:
train_err['datetime'] = pd.to_datetime(train_err.time.astype('str'),format='%Y%m%d%H%M%S')

In [279]:
train_prob['datetime'] = pd.to_datetime(train_prob.time.astype('str'),format='%Y%m%d%H%M%S')
train_qual['datetime'] = pd.to_datetime(train_qual.time.astype('str'),format='%Y%m%d%H%M%S')

#### EDA

In [285]:
train_err['user_id'].count()

16554663

In [289]:
len(train_prob['user_id'].unique()

5000

In [292]:
len(train_err.user_id.unique())

15000

In [290]:
len(train_prob)

5429

In [225]:
train_err[train_err.user_id==10000].sort_values(['datetime'])

user_id     317
time        317
model_nm    317
fwver       317
errtype     317
errcode     317
datetime    317
year        317
month       317
dtype: int64

In [238]:
dup = train_qual[train_qual.user_id==10000].duplicated()
train_qual[:len(dup)][ dup.apply(lambda x : True if x == False else False)]

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
6,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,4,0,0,0,0,4,0,0
12,20201130210000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,8,0,0
22,20201130210000,10000,05.15.2138,0.0,0,0.0,0,0,8,0,0,0,0,8,0,0


In [266]:
pd.set_option('display.max_row',500)

In [273]:
train_prob.sort_values(['user_id'])

,user_id,time
1116,10001,20201113190000
492,10004,20201104120000
3857,10005,20201113120000
3826,10006,20201110180000
414,10006,20201110170000
...,...,...
3870,24983,20201123230000
2378,24984,20201130150000
3829,24990,20201121170000
2799,24997,20201115150000


In [268]:
train_err[train_err.user_id==10000].sort_values(['datetime'])[:]

,user_id,time,model_nm,fwver,errtype,errcode,datetime,year,month
0,10000,20201101025616,model_3,05.15.2138,15,1,2020-11-01 02:56:16,2020,11
1,10000,20201101030309,model_3,05.15.2138,12,1,2020-11-01 03:03:09,2020,11
2,10000,20201101030309,model_3,05.15.2138,11,1,2020-11-01 03:03:09,2020,11
3,10000,20201101050514,model_3,05.15.2138,16,1,2020-11-01 05:05:14,2020,11
4,10000,20201101050515,model_3,05.15.2138,4,0,2020-11-01 05:05:15,2020,11
5,10000,20201101050516,model_3,05.15.2138,26,1,2020-11-01 05:05:16,2020,11
6,10000,20201101050522,model_3,05.15.2138,4,0,2020-11-01 05:05:22,2020,11
7,10000,20201101060903,model_3,05.15.2138,15,1,2020-11-01 06:09:03,2020,11
8,10000,20201101174754,model_3,05.15.2138,16,1,2020-11-01 17:47:54,2020,11
9,10000,20201101174756,model_3,05.15.2138,4,0,2020-11-01 17:47:56,2020,11


In [261]:
len(train_qual.user_id.unique())

8281

In [257]:
train_qual[train_qual.user_id==14300]
# train_qual[:len(dup)][ dup.apply(lambda x : True if x == False else False)]

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


In [235]:
dup = train_qual[train_qual.user_id==10001].duplicated()
train_qual[:len(dup)][ dup.apply(lambda x : True if x == False else False)]

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


In [217]:
train_err['month'] = train_err.datetime.dt.month


In [219]:
train_err['month'].unique()

array([11, 12, 10], dtype=int64)

In [177]:
pd.date_range('2010-01-21-01:03:55',periods=4)[0].second

55

In [56]:
pd.set_option('display.max_row',400)

In [65]:
pd.set_option('display.max_column',100)

In [66]:
pd.get_option('display.max_column')

100

In [54]:
pd.get_option('display.max_row')

200

In [58]:
id_100 = train_err[train_err.user_id==10000]

In [72]:
um =train_err.groupby(['user_id'])[['model_nm']].apply(lambda x : len(x['model_nm'].unique()) )

In [85]:
umc = um.value_counts()
umc.loc[2]/np.sum(umc )   

0.0468

In [89]:
usrg = train_err.groupby(['user_id'])

mean of rows for each user

In [102]:
np.sum(usrg['time'].apply(lambda x : len(x)) >25000)

21

In [105]:
len(usrg['time'])

15000

In [103]:
np.sum(usrg['time'].apply(lambda x : len(x)))

16554663

In [111]:
import time

1610181657.1060383

In [124]:
np.sum((train_err['time']%10000)>6000)

0

In [117]:
(train_err['time']%1000000).describe()

count    1.655466e+07
mean     1.245407e+05
std      6.980561e+04
min      0.000000e+00
25%      6.461800e+04
50%      1.304070e+05
75%      1.854190e+05
max      2.359590e+05
Name: time, dtype: float64

In [18]:
train_err['errcode'].unique()

array(['1', '0', '2', ..., '4526', '3965', '25999'], dtype=object)

In [10]:
train_err['model_nm'].value_counts()

model_1    5384491
model_0    4176279
model_2    3473254
model_3    1907438
model_4    1507802
model_8      36064
model_7      35356
model_5      32233
model_6       1746
Name: model_nm, dtype: int64

In [3]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [4]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [5]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:38<00:00, 427706.63it/s]


(15000, 42)

In [6]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

"\ntrain_err             = pd.read_csv(PATH+'train_err_data.csv')\ntrain_err['datetime'] = train_err['time'].apply(make_datetime)\n\nmin_day = train_err.datetime.min().date()\n\ntrain_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day\ntrain_err['days'] = train_err['days'].dt.days\ndisplay(train_err)\n\nid_error = train_err[['user_id','errtype','days']].values\nerror = np.zeros((train_user_number,42, 33))\nfor idx, err, days in tqdm(id_error):\n    error[idx - train_user_id_min,err - 1, days] += 1\nerror.shape\n"

## 2.15 train_quality


In [11]:
train_qual = pd.read_csv(PATH+'train_quality_data.csv')

In [14]:
train_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


## 2.2 problem

In [6]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

In [9]:
train_prob.sort_values(['time'])

,user_id,time
1084,18786,20201101000000
1455,21363,20201101010000
3192,14795,20201101010000
2488,17524,20201101010000
4925,11065,20201101020000
...,...,...
2760,22385,20201130220000
4149,11913,20201130220000
4589,12730,20201130220000
1425,13144,20201130230000


In [130]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [147]:
a = pd.DataFrame(np.random.randint(0,11,size=(10,3) )  )
b=pd.DataFrame(np.zeros(10))
c=np.zeros(10)
b[0][0]=4
c[b]=3

KeyError: 1

In [7]:
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

# 3. Light-gbm 모델 훈련

In [8]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = error
train_y = problem
del error, problem
print(train_x.shape)
print(train_y.shape)

(15000, 42)
(15000,)


In [9]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002308 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4353
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
[20]	valid_0's auc: 0.797482	valid_0's pr_auc: 0.79351
Early stopping, best iteration is:
[25]	valid_0's auc: 0.800713	valid_0's pr_auc: 0.798635
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001780 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4499
[LightGBM] [Info] Number of dat

# 4. 교차검증 점수 확인

In [10]:
print(np.mean(auc_scores))

0.7914130199022232


# 5. 제출 파일 생성

In [11]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [12]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [13]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:41<00:00, 395881.59it/s]

(14999, 42)


In [14]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [15]:
pred_ensemble

array([[0.57755635],
       [0.26089892],
       [0.31557557],
       ...,
       [0.38301962],
       [0.61611595],
       [0.34348301]])

In [16]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [17]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [18]:
sample_submssion.to_csv("dacon_baseline.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.577556
1,30001,0.260899
2,30002,0.315576
3,30003,0.535576
4,30004,0.454501
...,...,...
14994,44994,0.329001
14995,44995,0.315880
14996,44996,0.383020
14997,44997,0.616116
